In [1]:
import numpy as np
import pickle
import torch
import lightning.pytorch as pl
from tdc.benchmark_group import admet_group
from tqdm import tqdm
import chemprop
from chemprop import data, models, nn

In [2]:
pl.seed_everything(42)

Seed set to 42


42

In [3]:
benchmark_config = {
    'hia_hou': ('binary', False),
    'pgp_broccatelli': ('binary', False),
    'bioavailability_ma': ('binary', False),
    'bbb_martins': ('binary', False),
    'cyp2c9_veith': ('binary', False),
    'cyp2d6_veith': ('binary', False),
    'cyp3a4_veith': ('binary', False),
    'cyp2c9_substrate_carbonmangels': ('binary', False),
    'cyp2d6_substrate_carbonmangels': ('binary', False),
    'cyp3a4_substrate_carbonmangels': ('binary', False),
    'herg': ('binary', False),
    'ames': ('binary', False),
    'dili': ('binary', False)
}

In [4]:
group = admet_group(path='data/')
predictions = {}

100%|██████████████████████████████████████████████████████████████████████████| 1.47M/1.47M [00:00<00:00, 2.59MiB/s]
Extracting zip file...
Done!


In [5]:
for admet_benchmark in tqdm(benchmark_config.keys()):
    benchmark = group.get(admet_benchmark)
    name = benchmark['name']
    train_df, test_df = benchmark['train_val'], benchmark['test']
    train_pts = [
        data.MoleculeDatapoint.from_smi(smi, [y]) 
        for smi, y in zip(train_df['Drug'], train_df['Y'])
    ]
    test_pts = [
        data.MoleculeDatapoint.from_smi(smi, [y]) 
        for smi, y in zip(test_df['Drug'], test_df['Y'])
    ]

    train_dset = data.MoleculeDataset(train_pts)
    test_dset = data.MoleculeDataset(test_pts)

    train_loader = data.build_dataloader(train_dset, batch_size=64, shuffle=True, num_workers=0)
    test_loader = data.build_dataloader(test_dset, batch_size=64, shuffle=False, num_workers=0)

    mp = nn.BondMessagePassing()
    agg = nn.MeanAggregation()
    ffn = nn.BinaryClassificationFFN(n_tasks=1)
    
    model = models.MPNN(mp, agg, ffn, batch_norm=True)

    trainer = pl.Trainer(
        max_epochs=20,
        accelerator="cpu", 
        devices=1,
        enable_progress_bar=False,
        logger=False,
        enable_checkpointing=False
    )
    trainer.fit(model, train_loader)
    preds_batches = trainer.predict(model, test_loader)
    y_pred_proba = torch.cat(preds_batches).squeeze().cpu().numpy()

    predictions[f"{name}_y_pred_proba"] = y_pred_proba
    predictions[f"{name}_y_true"] = test_df['Y'].values

  0%|                                                                                         | 0/13 [00:00<?, ?it/s]GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
Loading `train_dataloader` to estimate number of stepping batches.


┏━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name            ┃ Type                    ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ message_passing │ BondMessagePassing      │  227 K │ train │     0 │
│ 1 │ agg             │ MeanAggregation         │      0 │ train │     0 │
│ 2 │ bn              │ BatchNorm1d             │    600 │ train │     0 │
│ 3 │ predictor       │ BinaryClassificationFFN │ 90.6 K │ train │     0 │
│ 4 │ X_d_transform   │ Identity                │      0 │ train │     0 │
│ 5 │ metrics         │ ModuleList              │      0 │ train │     0 │
└───┴─────────────────┴─────────────────────────┴────────┴───────┴───────┘

Trainable params: 318 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 318 K                                                                                                
Total estimated model params size (MB): 1                                                                          
Modules in train mode: 24                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

`Trainer.fit` stopped: `max_epochs=20` reached.
  8%|██████▏                                                                          | 1/13 [00:08<01:41,  8.48s/it]GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
Loading `train_dataloader` to estimate number of stepping batches.


┏━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name            ┃ Type                    ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ message_passing │ BondMessagePassing      │  227 K │ train │     0 │
│ 1 │ agg             │ MeanAggregation         │      0 │ train │     0 │
│ 2 │ bn              │ BatchNorm1d             │    600 │ train │     0 │
│ 3 │ predictor       │ BinaryClassificationFFN │ 90.6 K │ train │     0 │
│ 4 │ X_d_transform   │ Identity                │      0 │ train │     0 │
│ 5 │ metrics         │ ModuleList              │      0 │ train │     0 │
└───┴─────────────────┴─────────────────────────┴────────┴───────┴───────┘

...

Trainable params: 318 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 318 K                                                                                                
Total estimated model params size (MB): 1                                                                          
Modules in train mode: 24                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

`Trainer.fit` stopped: `max_epochs=20` reached.
 92%|█████████████████████████████████████████████████████████████████████████▊      | 12/13 [14:06<00:56, 56.77s/it]GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
Loading `train_dataloader` to estimate number of stepping batches.


┏━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name            ┃ Type                    ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ message_passing │ BondMessagePassing      │  227 K │ train │     0 │
│ 1 │ agg             │ MeanAggregation         │      0 │ train │     0 │
│ 2 │ bn              │ BatchNorm1d             │    600 │ train │     0 │
│ 3 │ predictor       │ BinaryClassificationFFN │ 90.6 K │ train │     0 │
│ 4 │ X_d_transform   │ Identity                │      0 │ train │     0 │
│ 5 │ metrics         │ ModuleList              │      0 │ train │     0 │
└───┴─────────────────┴─────────────────────────┴────────┴───────┴───────┘

Trainable params: 318 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 318 K                                                                                                
Total estimated model params size (MB): 1                                                                          
Modules in train mode: 24                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

`Trainer.fit` stopped: `max_epochs=20` reached.
100%|████████████████████████████████████████████████████████████████████████████████| 13/13 [14:15<00:00, 65.81s/it]


In [6]:
with open('chemprop_binary.pickle', 'wb') as handle:
    pickle.dump(predictions, handle, protocol=pickle.HIGHEST_PROTOCOL)